In [1]:
import pandas as pd
import numpy as np
import random

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score

from imblearn.over_sampling import SMOTE

In [2]:
X  = pd.read_csv("tornado-xtrain.csv")
xCols = X.columns
X = pd.DataFrame(preprocessing.scale(X))
X.columns = xCols
y = pd.read_csv("tornado-ytrain.csv", header=None, index_col=0)

# from feature importances as determined by decision tree
"""unimportantFeatures = ["EVENT_ID", "median_income", "population_density", "weekend", "average_range", "average_latitude", "average_longitude", "percent_land", "multi_vortex"]
X = X.drop(columns=unimportantFeatures)"""
X

/home/zaned/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  This is separate from the ipykernel package so we can avoid doing imports until


,EVENT_ID,median_income,population_density,housing_units_density,duration,weekend,sin_time,cos_time,sin_date,cos_date,tornado_intensity,tornado_area,average_range,minimum_range,average_latitude,average_longitude,percent_land,multi_vortex
0,-0.659569,1.098649,0.012169,-0.005170,-0.368472,-0.590965,-0.817780,0.075304,-0.530114,2.118623,-0.392406,-0.258449,-0.629740,-0.506841,-1.072970,0.235116,-0.023330,-0.071174
1,0.790458,0.428080,3.270950,3.242428,-0.368472,1.692148,1.606678,1.698298,0.446904,-0.749221,-0.392406,-0.233282,-0.493229,-0.506841,-0.899881,-0.615041,-0.060532,-0.071174
2,-1.431252,0.487838,-0.201964,-0.214536,-0.658824,1.692148,-0.763944,-0.218737,-1.027300,-1.008074,-0.392406,-0.269888,-0.356718,-0.231217,1.217137,-0.282695,0.411083,-0.071174
3,0.470828,3.065708,0.095714,0.109644,0.647761,-0.590965,-0.781959,-0.149646,1.048617,0.687161,-0.392406,-0.251585,0.189327,0.320031,0.329424,-3.656563,-0.328900,-0.071174
4,-1.333534,0.721748,-0.196180,-0.210999,0.792938,-0.590965,0.420625,1.766379,-0.007042,-0.999205,2.548380,0.347831,0.325838,0.320031,0.490067,-0.499896,0.398010,-0.071174
5,0.933694,3.497331,0.972811,0.911413,0.067057,1.692148,-0.726011,0.740710,-0.666618,2.112686,-0.392406,0.004654,-0.356718,-0.231217,-0.841492,-0.552762,-0.174891,-0.071174
6,-0.943042,-0.978364,-0.176258,-0.185270,0.647761,-0.590965,-0.416724,-0.844721,-1.290301,1.922912,2.548380,0.032108,2.373505,1.973774,-1.186515,1.208485,0.309809,-0.071174
7,-1.501964,-1.024805,-0.186284,-0.201029,1.228466,-0.590965,-0.795412,0.493317,0.510217,-0.696602,-0.392406,1.102822,-0.356718,-0.506841,-0.876804,1.216327,0.316014,-0.071174
8,-1.422917,0.667027,-0.175358,-0.188808,5.728926,1.692148,-0.773353,-0.184284,0.403238,-0.782436,2.548380,5.367372,-0.356718,-0.231217,1.131348,0.015214,0.386906,-0.071174
9,0.346118,-0.941912,-0.186412,-0.187200,-0.078119,-0.590965,2.142336,-0.949581,-1.429987,1.836254,-0.392406,-0.247009,-0.493229,-0.506841,-1.488670,0.838709,-2.600630,-0.071174


## KNN

In [3]:
knn = KNeighborsClassifier()

## RNN

In [4]:
rnn = RadiusNeighborsClassifier(outlier_label=0)

## Cross Validation Functions

In [5]:
def testClassifier(clf, X, y):
    y_pred = clf.predict(X)
    cm = confusion_matrix(y, y_pred)
    #cm_normalized = [cm[i]/sum(cm[i]) for i in range(len(cm))]
    f1 = f1_score(y, y_pred, average = 'macro')
    
    return cm, f1

In [6]:
def crossValidate(clf):
    kf = KFold(n_splits=10)
    cms = []
    f1s = []
    for train_fold, test_fold in kf.split(X):
        
        sm = SMOTE(sampling_strategy='not majority')
        X_train_balanced, y_train_balanced = sm.fit_resample(X.iloc[train_fold], y.iloc[train_fold].values.ravel())
    
        clf.fit(X_train_balanced, y_train_balanced.ravel())
        cm, f1 = testClassifier(clf, X.iloc[test_fold], y.iloc[test_fold])
        cms.append(cm)
        f1s.append(f1)
    
    return sum(cms), np.mean(f1)

## Testing Defaults

In [7]:
for c in [knn, rnn]:
    print(crossValidate(c))
    print()

(array([[8946, 1728,  131],
       [ 297,  395,  102],
       [   5,   59,   43]]), 0.40666778448468593)

(array([[10134,   647,    24],
       [  675,    96,    23],
       [   79,    14,    14]]), 0.3597932273185263)



## KNN Hyperparameter Optimization

* n_neighbors
* metric
* columns? TODO

In [8]:
krange = range(1, 15)

In [9]:
for k in krange:
    print(k)
    for m in ["euclidean", "manhattan"]:
        knn = KNeighborsClassifier(n_neighbors=k, metric=m)
        print(crossValidate(knn))

1
(array([[9796,  937,   72],
       [ 456,  266,   72],
       [  15,   54,   38]]), 0.41012548675502475)
(array([[10031,   717,    57],
       [  489,   236,    69],
       [   17,    52,    38]]), 0.4048092508210495)
2
(array([[9948,  791,   66],
       [ 481,  239,   74],
       [  15,   52,   40]]), 0.4130707459928044)
(array([[10133,   618,    54],
       [  505,   221,    68],
       [   19,    47,    41]]), 0.4490426304811455)
3
(array([[9286, 1417,  102],
       [ 355,  342,   97],
       [  10,   57,   40]]), 0.38308286818376985)
(array([[9532, 1197,   76],
       [ 396,  307,   91],
       [   9,   55,   43]]), 0.42858844022685827)
4
(array([[9423, 1278,  104],
       [ 371,  323,  100],
       [  11,   52,   44]]), 0.4047721065079728)
(array([[9657, 1072,   76],
       [ 405,  294,   95],
       [  10,   53,   44]]), 0.43582818440910637)
5
(array([[8964, 1712,  129],
       [ 302,  385,  107],
       [   5,   57,   45]]), 0.41121929453797756)
(array([[9215, 1492,   98],
   

## RNN Hyperparameter Optimization

* radius
* metric
* columns? TODO

In [10]:
rrange = np.arange(1.0, 10.0, 0.5)

In [11]:
for r in rrange:
    print(r)
    for m in ["euclidean", "manhattan"]:
        rnn = RadiusNeighborsClassifier(outlier_label=0, radius=r, metric=m)
        print(crossValidate(rnn))

1.0
(array([[10129,   653,    23],
       [  670,   101,    23],
       [   79,    14,    14]]), 0.35901607707495214)


/home/zaned/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zaned/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zaned/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zaned/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zaned/.local/lib/python3.6/site-packag

(array([[10755,    50,     0],
       [  788,     6,     0],
       [  106,     1,     0]]), 0.32061520260278026)
1.5
(array([[9205, 1499,  101],
       [ 482,  238,   74],
       [  57,   24,   26]]), 0.3607337150018514)


/home/zaned/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zaned/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zaned/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([[10677,   126,     2],
       [  777,    14,     3],
       [  101,     3,     3]]), 0.3266933735775395)
2.0
(array([[8886, 1720,  199],
       [ 355,  276,  163],
       [  29,   37,   41]]), 0.38498504735441497)
(array([[10519,   278,     8],
       [  736,    47,    11],
       [   92,     9,     6]]), 0.3518308762235578)
2.5
(array([[8963, 1540,  302],
       [ 293,  275,  226],
       [  18,   31,   58]]), 0.38130585582007726)
(array([[10264,   525,    16],
       [  685,    87,    22],
       [   81,    16,    10]]), 0.3568513563488438)
3.0
(array([[9196, 1190,  419],
       [ 289,  246,  259],
       [  14,   18,   75]]), 0.41551496036557767)
(array([[9951,  812,   42],
       [ 615,  142,   37],
       [  69,   20,   18]]), 0.3507496877608531)
3.5
(array([[9408,  946,  451],
       [ 284,  221,  289],
       [  11,   17,   79]]), 0.41921912953650614)
(array([[9600, 1137,   68],
       [ 540,  193,   61],
       [  54,   30,   23]]), 0.3439005927633954)
4.0
(array([[9501

## Best Model: RNN with radius=6.5